## Bases, truncation, and convergence

In [1]:
import scqubits as scq
import numpy as np

zp_yaml = """# zero-pi
branches:
- ["JJ", 1,2, EJ = 10, 20]
- ["JJ", 3,4, EJ, 20]
- ["L", 2,3, 0.008]
- ["L", 4,1, 0.008]
- ["C", 1,3, 0.02]
- ["C", 2,4, 0.02]
"""

zero_pi = scq.Circuit(zp_yaml, from_file=False)

### Basis choices

For **periodic degrees of freedom** scqubits employs the discrete charge basis.

For **extended degrees of freedom** two choices are available:
- `ext_basis = "discretized"` (default)<br> Discretizes the variables for extended degrees of freedom.
- `ext_basis = "harmonic"`<br>Employs the harmonic-oscillator basis for th extended degrees of freedom (taking the potential to be the one arising from the geometric inductance).

For instance, the following would switch the treatment of the extended degrees of freedom to using harmonic-oscillator basis:

`zero_pi = scq.Circuit(zp_yaml, from_file=False, ext_basis="harmonic")`

### Truncation, cutoffs and convergence


Each variable index comes with a **cutoff for basis truncation**. The corresponding attribute names can be listed like this:

In [2]:
zero_pi.cutoff_names

['cutoff_n_1', 'cutoff_ext_2', 'cutoff_ext_3']

These cutoffs must be varied by the user to ensure convergence. 
We distinguish two cutoff types:

#### Cutoffs for periodic degrees of freedom:
- `cutoff_n_<i>` - use all the integer charge states in the range `[-cutoff_n_<i>, cutoff_n_<i>]` for the periodic degree of freedom $i$.

#### Cutoffs for extended degrees of freedom:
- `cutoff_ext_<i>` - the meaning of this cutoff differs according to the choice of  basis, specified via `ext_basis` upon initialization of the `Circuit` instance.
    - `ext_basis = "discretized"`: <br>For discretization of $\theta_i$, `cutoff_ext_<i>` sets the number of points used to in the "spatial" range given by the attribute `discretized_phi_range`.
    - `ext_basis = "harmonic"`: <br>When using the harmonic oscillator basis, `cutoff_ext_<i>` sets the number of harmonic oscillator states admitted to the basis set for the extended degree of freedom $i$.

In [3]:
zero_pi.cutoff_n_1 = 5
zero_pi.cutoff_ext_2 = 10
zero_pi.cutoff_ext_3 = 10

All the discretized variable ranges can be viewed as:

In [4]:
zero_pi.discretized_phi_range

{2: (-18.84955592153876, 18.84955592153876), 3: (-18.84955592153876, 18.84955592153876)}

These are the default range settings $[-6\pi, 6\pi]$; discretized variable ranges can be changed with 

In [5]:
zero_pi.set_discretized_phi_range(var_indices=(2,3), phi_range=(-6*np.pi, 6*np.pi))

### Example: eigenenergies

Now, we can call `eigenvals()` to obtain **low-lying eigenenergies of the circuit Hamiltonian**: 

In [6]:
zero_pi.eigenvals()

array([-13.25816395, -13.25645441, -13.24363991, -13.24193037,
       -12.134373  , -12.13436007])

Increasing the above cutoff values reveals that these eigenvalues have not converged yet. Increasing cutoff values increases the Hilbert space dimension and, thus, increases memory requirements and runtime. A strategy that can help mitigate this problem is the use of [Hierarchical diagonalization](./custom_circuit_hd.ipynb).


## Chosing basis states for each of the subsystems
It is also possible to chose a basis states for each of the subsystems independently.
We start with defining a particular circuit

In [14]:
full_yaml = f"""branches:
- [JJ, 0, 1, 10, 2.5]
- [L, 2, 1, 0.5]
- [L, 0 , 2 , 10]
### coupling element
- [C, 2, 3, 5]
### tunable resonator
- [JJ, 0, 3, 50, 2]
- [C, 3, 4, 0.5]
- [L, 4, 0, 8]
"""
circ = scq.Circuit(full_yaml, from_file=False, ext_basis="discretized")

and see its internal structure 

In [16]:
circ.info()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

we can now adjust the hierarchy of diefferent degrees of freedom, while defining a particular basis set for each

In [17]:
circ = scq.Circuit(full_yaml, from_file=False, ext_basis="discretized")
trans_mat = np.array([[1, 0, 0, 0], 
                      [0, 1, 0, 0], 
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])
circ.configure(transformation_matrix=trans_mat, closure_branches=[circ.branches[0], circ.branches[-1]]) # chosing a transformation matrix, and closure branches for the two loops.
circ._configure(system_hierarchy=[[[1], [2]], [3, 4]], subsystem_trunc_dims=[[10, [10, 10]], 20], ext_basis=[["harmonic", "discretized"], "discretized"])

and see the updated circuit detailed information

In [18]:
circ.info()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>